In [1]:
import pandas as pd
import warnings
from pandas import ExcelWriter
import csv

warnings.simplefilter(action = "ignore")

data = pd.read_csv("Final Data.csv", encoding = "iso-8859-1")
data = data[data['Include in Data Analysis'] == 1]

to_numeric_list = ['Site EUI (kBtu/ft2)',
                    'Site Energy Use (kBtu)',
                    'Source Energy Use (kBtu)',
                    'Source EUI (kBtu/ft2)',
                    'Property GFA - EPA Calculated (Buildings) (ft2)',
                   'Electricity Use - Grid Purchase and Generated from Onsite Renewable Systems (kWh)',
                    'Natural Gas Use (kBtu)',
                   'Total GHG Emissions (Metric Tons CO2e)']    
        
for col in data:
    if col in to_numeric_list:
        data[col] = pd.to_numeric(data[col], errors = 'coerce')
    else:
        pass

In [2]:
categories = list(set(data['Primary Property Type Category - Reduced']))
epa_cat = list(set(data['Primary Property Type - EPA Calculated']))

In [3]:
'''
1. ReducedSiteEui = Difference between EUI and Median
2. TotalKBTUs = Multiply Difference by Gross Floor Area
3. PercentageReduction = Result from Step 2 / Site EUI (not weather normalized)
4. Ratio Between Source and Site Energy Use
5. Step 2 - reduction multiplied by Step 4 (tells you reduction in source energy use)
6. Reduction in Electricity - Takes percentage reduction (Step 3) and if not 0, multiplies by 
        Electricy Grid Use Purchase and Generated from onsite renewable systems 
        Multiplies percentage reduction by B31 - Conversion Rate/number for KWH per KBT - 0.293083 (constant)
7. Reduction in Natural Gas - Multiply Step 1 by Natural gas use KTBU
8. GHG reductions - Reduction EUI (Step 1) Times Total GHG emissions Metric Tons co2e
9. Reduction of 2% of EUI for top performers
'''

'\n1. ReducedSiteEui = Difference between EUI and Median\n2. TotalKBTUs = Multiply Difference by Gross Floor Area\n3. PercentageReduction = Result from Step 2 / Site EUI (not weather normalized)\n4. Ratio Between Source and Site Energy Use\n5. Step 2 - reduction multiplied by Step 4 (tells you reduction in source energy use)\n6. Reduction in Electricity - Takes percentage reduction (Step 3) and if not 0, multiplies by \n        Electricy Grid Use Purchase and Generated from onsite renewable systems \n        Multiplies percentage reduction by B31 - Conversion Rate/number for KWH per KBT - 0.293083 (constant)\n7. Reduction in Natural Gas - Multiply Step 1 by Natural gas use KTBU\n8. GHG reductions - Reduction EUI (Step 1) Times Total GHG emissions Metric Tons co2e\n9. Reduction of 2% of EUI for top performers\n'

In [4]:
def calculate_reductions(df, identifier, to_twenty_fifth_percentile=False, top_quartile_reduce_2_perc=False):
    '''
    Inputs:
        df = pandas dataframe containing data (subset by performance or property type, etc.)
        identifier = string to add to column names (e.g. 'by median')
        to_median = Boolean (True if we are calculating Site EUI median, False otherwise)
        to_twenty_fifth_percentile = Boolean (True if we are calculating Site EUI 75th percentile, False otherwise)
        top_quartile_reduce_2_perc = Boolean (True if we are calculating 2% reduction on Top Quartile, False otherwise)
    Outputs: 
        pandas dataframe with new columns
    '''
    #Constant
    KWH_TO_KBTU_CONVERSION = 0.293083
    
    
    #Creating cut-offs for different scenarios
    if to_twenty_fifth_percentile:
        #use twenty fifth percentile
        cut_off = df['Site EUI (kBtu/ft2)'].quantile(.25)
    elif top_quartile_reduce_2_perc:
        #use best 25% of performers in Site energy use as cut-off
        cut_off = df['Site Energy Use (kBtu)'].quantile(.25)
    else:
        #defaults to using median
        cut_off = df['Site EUI (kBtu/ft2)'].median()

    #for reducing top quartile by 2%
    if top_quartile_reduce_2_perc:
        #first column
        #NOTE: we perform the 2% reduction on the best 25% of buildings measured by Site Energy Use, not Site EUI. 
        #Site EUI is used for the to_median and to_twenty_fifth_percentile calculations.
        reduced_energy = df[df['Site Energy Use (kBtu)'] < cut_off]['Site Energy Use (kBtu)'] * 0.02
        df['reduced site eui ' + '(' + identifier +')'] = (
           reduced_energy / df['Property GFA - EPA Calculated (Buildings) (ft2)'])
    else:
        #first column: 
        df['reduced site eui ' + '(' + identifier +')'] = df[data['Site EUI (kBtu/ft2)'] > cut_off].apply(
            lambda row: row['Site EUI (kBtu/ft2)'] - cut_off, axis=1)
        
    # second column
    df['total reduced site energy (kBtu) '+ '(' + identifier +')'] = (
        df['Property GFA - EPA Calculated (Buildings) (ft2)'] * 
        df['reduced site eui ' + '(' + identifier +')'])
    # third column: PercentageReduction = Result from Step 2 / Site EUI (not weather normalized)
    df['percentage reduction in energy use '+ '(' + identifier +')'] = (
        df['total reduced site energy (kBtu) '+ '(' + identifier +')'] / 
        df['Site Energy Use (kBtu)'])
    # fourth column
    df['ratio of source to site EUI '+ '(' + identifier +')'] = (
        df['Source Energy Use (kBtu)'] / 
        df['Site Energy Use (kBtu)'])
    # fifth column: total reduce source energy use
    df['total reduced source (kBtu) '+ '(' + identifier +')'] = (
        df['total reduced site energy (kBtu) '+ '(' + identifier +')'] * 
        df['ratio of source to site EUI '+ '(' + identifier +')'])
    # sixth column: reduction in electricy use
    df['reduction in kwh/year '+ '(' + identifier +')'] = (
        df['percentage reduction in energy use '+ '(' + identifier +')'] *
        df['Electricity Use - Grid Purchase and Generated from Onsite Renewable Systems (kWh)'] * 
        KWH_TO_KBTU_CONVERSION)
    # seventh column: reduction in site natural gas
    df['natural gas reduction (kBTUs/year) '+ '(' + identifier +')'] = (
        df['percentage reduction in energy use '+ '(' + identifier +')'] * 
        df['Natural Gas Use (kBtu)'])
    # eighth column: reduction in greenhouse gas emissions
    df['GHG reduction (MT CO2e reduced / year) '+ '(' + identifier +')'] = (
            df['percentage reduction in energy use '+ '(' + identifier +')'] * 
            df['Total GHG Emissions (Metric Tons CO2e)'])
    
    return df

In [5]:
'''
Healthcare Categories
    - Ambulatory, Output Rehabilition, Urgent Care
    - Hospital
    - Medical Office
    - Other - Specialty Hospital
    
'''
ambulatory_output_urgentcare_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Ambulatory Surgical Center') |
    (data['Primary Property Type - EPA Calculated'] == 'Outpatient Rehabilitation/Physical Therapy') | 
    (data['Primary Property Type - EPA Calculated'] == 'Urgent Care/Clinic/Other Outpatient')]

hospital_df = data[
    data['Primary Property Type - EPA Calculated'] == 'Hospital (General Medical & Surgical)']

medical_df = data[data['Primary Property Type - EPA Calculated'] == 'Medical Office']

other_hospital_df = data[data['Primary Property Type - EPA Calculated'] == 'Other - Specialty Hospital']

In [6]:
'''
Lodging Categories
    - Hotel
    - Other - Lodging and Residence Hall
    - Senior Care
'''
hotel_df = data[(data['Primary Property Type - EPA Calculated'] == 'Hotel')]

other_res_hall_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Other - Lodging/Residential') |
    (data['Primary Property Type - EPA Calculated'] == 'Residence Hall/Dormitory')]

senior_care_df = data[data['Primary Property Type - EPA Calculated'] == 'Senior Care Community']

In [7]:
'''
Office Categories
    - Financial Office
    - Small Office: < 100,000 square feet
    - Medium Office: 100,000 <= office sqft < 250,000
    - Large Office: >= 250,000
'''
financial_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Financial Office')]

small_office_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Office') &
    (data['Property GFA - EPA Calculated (Buildings) (ft2)'] < 100000)]

med_office_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Office') &
    (data['Property GFA - EPA Calculated (Buildings) (ft2)'] >= 100000) & 
    (data['Property GFA - EPA Calculated (Buildings) (ft2)'] < 250000)]

large_office_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Office') &
    (data['Property GFA - EPA Calculated (Buildings) (ft2)'] >= 250000)]

In [8]:
'''
Retail Categories
    - Automobile Dealership
    - Enclosed & Other Mall
    - Retail Store
    - Strip Mall
    - Supermarket/Grocery Store
    - Wholesale Club/Other - Services/Repair - Services
'''
auto_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Automobile Dealership')]

enclosed_mall_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Enclosed Mall') |
    (data['Primary Property Type - EPA Calculated'] == 'Other - Mall')]

retail_store_df = data[data['Primary Property Type - EPA Calculated'] == 'Retail Store']

strip_mall_df = data[data['Primary Property Type - EPA Calculated'] == 'Strip Mall']

supermarket_df = data[data['Primary Property Type - EPA Calculated'] == 'Supermarket/Grocery Store']

wholesale_other_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Wholesale Club/Supercenter') |
    (data['Primary Property Type - EPA Calculated'] == 'Other - Services') |
    (data['Primary Property Type - EPA Calculated'] == 'Repair Services (Vehicle, Shoe, Locksmith, etc.)')]

In [9]:
'''
K12 Categories
    - Small School: < 100,000 square feet
    - Medium School: 100,000 <= office sqft < 250,000
    - Large School: >= 250,000
'''

small_school_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'K-12 School') &
    (data['Property GFA - EPA Calculated (Buildings) (ft2)'] < 100000)]

med_school_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'K-12 School') &
    (data['Property GFA - EPA Calculated (Buildings) (ft2)'] >= 100000) & 
    (data['Property GFA - EPA Calculated (Buildings) (ft2)'] < 250000)]

large_school_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'K-12 School') &
    (data['Property GFA - EPA Calculated (Buildings) (ft2)'] >= 250000)]

In [10]:
'''
Multifamily Categories
    - Small Multifamily: < 500,000 square feet
    - Medium Multifamily: 500,000 <= office sqft < 1,000,000
    - Large Multifamily: >= 1,000,000
'''

small_multifamily_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Multifamily Housing') &
    (data['Property GFA - EPA Calculated (Buildings) (ft2)'] < 500000)]

med_multifamily_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Multifamily Housing') &
    (data['Property GFA - EPA Calculated (Buildings) (ft2)'] >= 500000) & 
    (data['Property GFA - EPA Calculated (Buildings) (ft2)'] < 1000000)]

large_multifamily_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Multifamily Housing') &
    (data['Property GFA - EPA Calculated (Buildings) (ft2)'] >= 1000000)]

In [11]:
'''
Other Categories
    - Adult Education/Other Ed/Preschool
    - Convention Center/Other Entertainment/Public Assembly
    - Court/Other
    - Distribution Center/Other
    - Fitness
    - Indoor Arena/Other
    - Laboratory
    - Library
    - Mixed Used
    - Movie Theather/Entertainment
    - Museum
    - Worship Facility
'''

adult_other_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Adult Education') |
    (data['Primary Property Type - EPA Calculated'] == 'Other - Education') |
    (data['Primary Property Type - EPA Calculated'] == 'Pre-school/Daycare')]

convention_other_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Convention Center') |
    (data['Primary Property Type - EPA Calculated'] == 'Other - Entertainment/Public Assembly')]

court_other_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Courthouse') |
    (data['Primary Property Type - EPA Calculated'] == 'Other - Public Services') |
    (data['Primary Property Type - EPA Calculated'] == 'Prison/Incarceration')]

distribution_other_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Distribution Center') |
    (data['Primary Property Type - EPA Calculated'] == 'Other') |
    (data['Primary Property Type - EPA Calculated'] == 'Not Available')|
    (data['Primary Property Type - EPA Calculated'] == 'Self-Storage Facility')]

fitness_other_df = data[
    data['Primary Property Type - EPA Calculated'] == 'Fitness Center/Health Club/Gym']

indoor_other_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Indoor Arena') |
    (data['Primary Property Type - EPA Calculated'] == 'Other - Recreation')]

laboratory_df = data[data['Primary Property Type - EPA Calculated'] == 'Laboratory']

library_df = data[data['Primary Property Type - EPA Calculated'] == 'Library']

mixed_df = data[data['Primary Property Type - EPA Calculated'] == 'Mixed Use Property']

movie_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'Movie Theater') |
    (data['Primary Property Type - EPA Calculated'] == 'Performing Arts') |
    (data['Primary Property Type - EPA Calculated'] == 'Social/Meeting Hall')]

museum_df = data[data['Primary Property Type - EPA Calculated'] == 'Museum']

worship_df = data[data['Primary Property Type - EPA Calculated'] == 'Worship Facility']

In [12]:
'''
University Categories
    - Small University: < 100,000 square feet
    - Medium University: 100,000 <= office sqft < 250,000
    - Large University: >= 250,000
'''

small_uni_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'College/University') &
    (data['Property GFA - EPA Calculated (Buildings) (ft2)'] < 100000)]

med_uni_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'College/University') &
    (data['Property GFA - EPA Calculated (Buildings) (ft2)'] >= 100000) & 
    (data['Property GFA - EPA Calculated (Buildings) (ft2)'] < 250000)]

large_uni_df = data[
    (data['Primary Property Type - EPA Calculated'] == 'College/University') &
    (data['Property GFA - EPA Calculated (Buildings) (ft2)'] >= 250000)]

In [13]:
# list of dataframes of EPA property type categories. Created this in order to iterate through when applying calculation func.
df_list = ([small_uni_df, med_uni_df, large_uni_df, ambulatory_output_urgentcare_df, hospital_df, medical_df, 
            other_hospital_df, hotel_df, other_res_hall_df, senior_care_df, financial_df, small_office_df, med_office_df, 
            large_office_df, auto_df, enclosed_mall_df, retail_store_df, strip_mall_df, supermarket_df, wholesale_other_df, 
            small_school_df, med_school_df, large_school_df, small_multifamily_df, med_multifamily_df, large_multifamily_df, 
           adult_other_df, convention_other_df, court_other_df, distribution_other_df, fitness_other_df, indoor_other_df, 
            laboratory_df, library_df, mixed_df, movie_df, museum_df, worship_df])

# For loop for running the calculate_reductions function with median site EUI.
for df in df_list:
    calculate_reductions(df, 'to Median',)

# For loop for running the calculate_reductions function with 75th percentile site EUI.
for df in df_list:
    calculate_reductions(df, 'to Top Quartile', to_twenty_fifth_percentile = True)

# For loop for running the calculate_reductions function with 75th percentile site EUI.
for df in df_list:
    calculate_reductions(df, 'Top Performers', top_quartile_reduce_2_perc = True)

In [14]:
category_dict = {'university_list': [small_uni_df, med_uni_df, large_uni_df],
                'healthcare_list': [ambulatory_output_urgentcare_df, hospital_df, medical_df, other_hospital_df],
                'lodging_list': [hotel_df, other_res_hall_df, senior_care_df],
                'office_list': [financial_df, small_office_df, med_office_df, large_office_df],
                'retail_list': [auto_df, enclosed_mall_df, retail_store_df, strip_mall_df, supermarket_df, wholesale_other_df],
                'school_list': [small_school_df, med_school_df, large_school_df],
                'multifamily_list': [small_multifamily_df, med_multifamily_df, large_multifamily_df],
                'other_list': [adult_other_df, convention_other_df, court_other_df, distribution_other_df, 
                                fitness_other_df, indoor_other_df,
                                laboratory_df, library_df, mixed_df, movie_df, museum_df, worship_df]
                 }

for name in category_dict:
    writer = ExcelWriter('Analysis 40 {0} Results.xlsx'.format(name))
    for n, df in enumerate(category_dict[name]):
        df.to_excel(writer,'sheet%s' % n)
    writer.save()

In [78]:
def calculate_row(df, scenario):
    '''
    '''
    row_index = {
        'low': 'Low End: All properties below the 50th percentile reach the 50th percentile',
        'mid': 'High End: All properties below the 75th percentile reach the 75th percentile',
        'top': 'High Performers (Top 25th Percentile) Reduce Site Energy Use by 2%',
        'energy-low': 'Energy not reduced in Low-End Scenario',
        'energy-high': 'Energy not reduced in High-End Scenario'}
    
    source_energy_reductions = 'Source Energy Savings (MMBTU/year)'
    site_energy_reductions = 'Site Energy Savings/year (MMBTU/year)'
    GHG_emissions_reductions = 'GHG Emission Reductions (metric tons CO2e/year)'
    cost_savings_elec = 'Energy Cost Savings ($/year) (for electricity)'
    cost_savings_ng = 'Energy Cost Savings ($/year) (for natural gas)'
    investment = 'Estimated Investment'
    jobs ='Jobs Created'
    number_of_buildings = 'Number of Buildings Included in Savings'
        
    label_dict = {
            'low': {
                    source_energy_reductions: 'total reduced source (kBtu) (to Median)',
                    site_energy_reductions: 'total reduced site energy (kBtu) (to Median)',
                    GHG_emissions_reductions: 'GHG reduction (MT CO2e reduced / year) (to Median)',
                    cost_savings_elec: 'reduction in kwh/year (to Median)',
                    cost_savings_ng: 'natural gas reduction (kBTUs/year) (to Median)',
                    number_of_buildings: 'reduced site eui (to Median)'},
            'mid': {
                    source_energy_reductions: 'total reduced source (kBtu) (to Top Quartile)',
                    site_energy_reductions: 'total reduced site energy (kBtu) (to Top Quartile)',
                    GHG_emissions_reductions: 'GHG reduction (MT CO2e reduced / year) (to Top Quartile)',
                    cost_savings_elec: 'reduction in kwh/year (to Top Quartile)',
                    cost_savings_ng: 'natural gas reduction (kBTUs/year) (to Top Quartile)',
                    number_of_buildings: 'reduced site eui (to Top Quartile)'},
            'top': {
                    source_energy_reductions: 'total reduced source (kBtu) (Top Performers)',
                    site_energy_reductions: 'total reduced site energy (kBtu) (Top Performers)',
                    GHG_emissions_reductions: 'GHG reduction (MT CO2e reduced / year) (Top Performers)',
                    cost_savings_elec: 'reduction in kwh/year (Top Performers)',
                    cost_savings_ng: 'natural gas reduction (kBTUs/year) (Top Performers)',
                    number_of_buildings: 'reduced site eui (Top Performers)'}
        }

        
    CONSTANTS = {
                'CO2e per car / year': 4750, #2013
                'Commercial Electricity Price': 0.076, #2016 Estimate based on Adjusted Illinois State Avg Energy Prices
                'Conversion kBTU per 1000 ft3': 1021, #http://www.eia.gov/dnav/ng/ng_cons_heat_a_epg0_vgth_btucf_a.htm
                'Residential NG Price':  10.3325, #2016 Estimate based on Adjusted Illinois State Avg Energy Prices
                'Estimated Payback Period': 3.5, #Shaw Environmental & Infrastructure Group, based on average commercial EE project payback period
                'Labor Share of Investment': .5, #Shaw Environmental & Infrastructure Group

                'Average Salary': 70000 #Est. ($56,140 = average annual wage, nonresidential building construction, all occupations; http://www.bls.gov/oes/current/naics4_236200.htm) 
                }
    
    if scenario == 'energy-low':
        calculation_dict = {
                'Source Energy Savings (MMBTU/year)': (
                    df['Source Energy Use (kBtu)'].sum() / 1000
                    ) - (df[label_dict['low'][source_energy_reductions]].sum() / 1000),
                'Site Energy Savings/year (MMBTU/year)': (
                    df['Site Energy Use (kBtu)'].sum() / 1000
                    ) - (df[label_dict['low'][site_energy_reductions]].sum() / 1000)}
    elif scenario == 'energy-high':
        calculation_dict = {
        'Source Energy Savings (MMBTU/year)': (
            df['Source Energy Use (kBtu)'].sum() / 1000
            ) - (df[label_dict['mid'][source_energy_reductions]].sum() / 1000),
        'Site Energy Savings/year (MMBTU/year)': (
            df['Site Energy Use (kBtu)'].sum() / 1000
            ) - (df[label_dict['mid'][site_energy_reductions]].sum() / 1000)}
    else:
        calculation_dict = {
                    'Source Energy Savings (MMBTU/year)': (
                        df[label_dict[scenario][source_energy_reductions]].sum() / 1000
                        ),
                    'Site Energy Savings/year (MMBTU/year)': (
                        df[label_dict[scenario][site_energy_reductions]].sum() / 1000
                        ),
                    'GHG Emission Reductions (metric tons CO2e/year)': (
                        df[label_dict[scenario][GHG_emissions_reductions]].sum()
                        ),
                    'GHG Emission Reductions - Car equivalent / year': (
                        df[label_dict[scenario][GHG_emissions_reductions]].sum() * 1000 / 
                           CONSTANTS['CO2e per car / year']
                        ),
                    'Energy Cost Savings ($/year) (for electricity and natural gas only)': (
                        (df[label_dict[scenario][cost_savings_elec]].sum() * CONSTANTS['Commercial Electricity Price']) + 
                        (df[label_dict[scenario][cost_savings_ng ]].sum() /
                         CONSTANTS['Conversion kBTU per 1000 ft3'] * 
                         CONSTANTS['Residential NG Price'])
                        ),
                    'Estimated Investment': (
                        (   #Same as 'Energy Cost Savings ($/year) (for electricity and natural gas only)' multiplied by Payback Period
                            (df[label_dict[scenario][cost_savings_elec]].sum() * CONSTANTS['Commercial Electricity Price']) + 
                            (df[label_dict[scenario][cost_savings_ng ]].sum() /
                             CONSTANTS['Conversion kBTU per 1000 ft3'] * 
                             CONSTANTS['Residential NG Price'])
                        ) * CONSTANTS['Estimated Payback Period']
                        ),
                    'Jobs Created': (
                        (
                            (#Same as 'Energy Cost Savings ($/year) (for electricity and natural gas only)' multiplied by Payback Period
                                (df[label_dict[scenario][cost_savings_elec]].sum() * CONSTANTS['Commercial Electricity Price']) + 
                                (df[label_dict[scenario][cost_savings_ng ]].sum() /
                                 CONSTANTS['Conversion kBTU per 1000 ft3'] * 
                                 CONSTANTS['Residential NG Price'])
                             ) * CONSTANTS['Estimated Payback Period']) *
                            CONSTANTS['Labor Share of Investment'] /
                            CONSTANTS['Average Salary']
                        ),
                    'Number of Buildings Included in Savings': df[label_dict[scenario][number_of_buildings]].count()
                    }

    return pd.DataFrame(calculation_dict, index=[row_index[scenario]])


def produce_savings_results(df_list, name):
    '''
    '''
    scenarios = ['mid', 'top', 'energy-low', 'energy-high']
    writer = ExcelWriter('Analysis 40 {0} Tables.xlsx'.format(name))
    for n, df in enumerate(df_list):
        results = calculate_row(df, 'low')
        for scenario in scenarios:
            results = results.append(calculate_row(df, scenario))
        results.to_excel(writer,'sheet%s' % n)
    writer.save()
        
for category in category_dict:
    produce_savings_results(category_dict[category], category)

In [76]:
calculate_row(other_hospital_df, 'low')

,Energy Cost Savings ($/year) (for electricity and natural gas only),Estimated Investment,GHG Emission Reductions (metric tons CO2e/year),GHG Emission Reductions - Car equivalent / year,Jobs Created,Number of Buildings Included in Savings,Site Energy Savings/year (MMBTU/year),Source Energy Savings (MMBTU/year)
Low End: All properties below the 50th percentile reach the 50th percentile,35023.936385,122583.777347,437.525937,92.110724,0.875598,2,4001.1855,7383.000336


In [71]:
other_hospital_df['Site EUI (kBtu/ft2)']

72       38.0
78      177.4
1376    110.5
1414    159.8
1536    185.5
Name: Site EUI (kBtu/ft2), dtype: float64

In [48]:
med_uni_df['reduced site eui (Top Performers)'].count()

11

{'apple': 'a'}